In [3]:
pip install gradio rasterio scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
import joblib

model_path = '/content/drive/MyDrive/best_rf_model.pkl'
model = joblib.load(model_path)

def predict_light_pollution(file, real_value=None):
    # Extrage anul și luna din numele fișierului
    filename = file.name
    parts = filename.split('_')  # Numele fișierului ar trebui să fie de forma viirs_YYYY_MM.tif
    year = int(parts[1])  # Anul
    month = int(parts[2].split('.')[0])  # Luna (partea din nume înainte de .tif)

    # Prelucrarea imaginii și realizarea predicției
    image = Image.open(file.name).convert('L')  # Convertim imaginea la grayscale (L)

    if image is None:
        return "Eroare la încărcarea imaginii"

    image = image.resize((100, 100))  # Redimensionează imaginea
    image = np.array(image)  # Convertim imaginea într-un array numpy
    intensity = np.mean(image)  # Calculăm media intensității pixelilor

    # Realizează predicția folosind anul și luna
    prediction = model.predict([[year, month]])  # Anul și luna ca intrări pentru model

    if real_value is not None:
        # Calculează eroarea de predicție
        error = abs(prediction[0] - real_value)  # Diferența absolută între valoarea reală și prezisă
        return f"Predicție poluare luminoasă pentru {year}, luna {month}: {prediction[0]:.2f}. Eroare: {error:.2f}"
    else:
        return f"Predicție poluare luminoasă pentru {year}, luna {month}: {prediction[0]:.2f}"

# Creează interfața Gradio cu un câmp opțional pentru valoarea reală
ui = gr.Interface(fn=predict_light_pollution,
                  inputs=[gr.File(label="Încărcați imaginea VIIRS (.tif)"),
                          gr.Number(label="Valoare reală poluare (opțional)")],
                  outputs="text",
                  title="Predicție Poluare Luminoasă")

# Lansează aplicația
ui.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1b93eed1f7330c2a88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [8]:
import os

# Verifică dacă fișierul există
file_path = '/content/drive/MyDrive/best_rf_model.pkl'
print(os.path.exists(file_path))  # Ar trebui să returneze True dacă fișierul există


True
